In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor

from sklearn.metrics import root_mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import joblib

In [2]:
df = pd.read_parquet('../04_EDA/car_ads_model.parquet')

In [3]:
df.shape

(182826, 20)

In [4]:
model_counts = df[['brand','model']].value_counts().reset_index()

In [6]:
filtered_models = model_counts[model_counts['count'] > 20][['brand','model']]

In [8]:
df = df.merge(filtered_models, how='inner', on=['brand','model'])

In [9]:
df.shape

(178897, 20)

In [10]:
# Step 2: Lists to store results
all_results = []
performance_metrics = []

In [22]:
# Function to encode 'cv' (horsepower) column
def encode_cv(group, max_unique_cv=5):
    if group['cv'].nunique() <= max_unique_cv:
        # One-hot encoding if the number of unique 'cv' values is small
        return pd.get_dummies(group['cv'], prefix='cv')
    else:
        # Bin and label-encode if there are too many unique values
        bins = pd.qcut(group['cv'], q=5, duplicates='drop', labels=False)  # Adjust 'q' for the number of bins
        return bins

# Function to encode 'fuel' and other categorical variables
def encode_categorical_features(group):
    # One-hot encoding for 'fuel'
    fuel_encoded = pd.get_dummies(group['fuel'], prefix='fuel')
    
    # Handle CV encoding if 'cv' is present
    if group['cv'].notnull().all():
        cv_encoded = encode_cv(group)
    else:
        cv_encoded = pd.DataFrame(index=group.index)  # Empty if no 'cv'
    
    # Combine encoded features
    return pd.concat([fuel_encoded, cv_encoded], axis=1)

# Function to select a model based on the number of samples
def select_model(n_samples):
    if n_samples < 30:
        return SVR(kernel='poly', degree=2, C=0.2)  # L2 Regularization
    elif n_samples < 2_000:
        return XGBRegressor(n_estimators=500, learning_rate=0.05, max_depth=7)  # Tree-based regularization
    else:
        return RandomForestRegressor(n_estimators=500, max_depth=7)  # Tree-based with max depth

# Function to choose and apply model
def choose_model(group):
    X = group[['km', 'age_years', 'is_automatic']]  # Base features
    
    # Encode categorical features (fuel and cv)
    cat_encoded = encode_categorical_features(group)
    X = pd.concat([X, cat_encoded], axis=1)

    y = group['price']  # Target (Price)
    n_samples = len(group)
    
    # If very few samples, predict the median price
    if n_samples < 5:
        group['predicted_price'] = [group['price'].median()] * n_samples
        group['price_diff'] = group['price'] - group['predicted_price']
        rmse = root_mean_squared_error(y, group['predicted_price'])
        mape = mean_absolute_percentage_error(y, group['predicted_price'])
        performance_metrics.append({
            'brand': group['brand'].iloc[0],
            'model': group['model'].iloc[0],
            'rmse': rmse,
            'mape': mape
        })
        all_results.append(group)
        return

    # Train-test split logic
    test_size = 0.3 if n_samples >= 40 else 0.4
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

    # Select the model based on dataset size
    model = select_model(n_samples)
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Predicting on test set
    y_pred_test = model.predict(X_test)
    
    # Train on the entire dataset to predict for all data
    model.fit(X, y)
    y_pred_all = model.predict(X)

    # Save the trained model to disk
    brand = group['brand'].iloc[0]
    model_name = group['model'].iloc[0]
    model_file_name = f'saved_models/{brand}_{model_name}.pkl'
    joblib.dump(model, model_file_name)
    
    # Calculate performance metrics
    rmse = root_mean_squared_error(y_test, y_pred_test)
    mape = mean_absolute_percentage_error(y_test, y_pred_test) * 100
    performance_metrics.append({
        'brand': brand,
        'model': model_name,
        'rmse': round(rmse, 2),
        'mape': round(mape, 2)
    })

    # Add predictions for all data points
    group['predicted_price'] = y_pred_all
    group['price_diff'] = group['price'] - group['predicted_price']
    
    all_results.append(group)

In [23]:
# Apply the function to each group of brand-model
df.groupby(['brand', 'model']).apply(choose_model, include_groups=True)

C:\Users\samue\AppData\Local\Temp\ipykernel_8652\1134254477.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby(['brand', 'model']).apply(choose_model, include_groups=True)


""


In [24]:
results_df = pd.DataFrame(performance_metrics)  # Model performance metrics
final_results_df = pd.concat(all_results, ignore_index=True)  # Test data with predictions and price diff

In [25]:
final_cols_order = ['price','predicted_price', 'price_diff', 'brand', 'model', 'is_automatic','km', 'fuel',
                    'year', 'age_years', 'cv', 'kw', 'body_type', 'cubic_capacity', 'is_seller_pro','is_certified', 'offer_type',
                    'location', 'ad_days_creation', 'ad_days_published', 'title']

In [26]:
final_results_df = final_results_df[final_cols_order]

numeric_cols_list = final_results_df.select_dtypes(include='number').columns.values
final_results_df[numeric_cols_list] =  final_results_df[numeric_cols_list].round(0).astype('Int32')


In [27]:
results_df['rmse'].describe(percentiles=np.arange(0,1.1,0.1)).astype(int)

count     1240
mean      3938
std       3572
min        456
0%         456
10%       1273
20%       1528
30%       1845
40%       2197
50%       2764
60%       3324
70%       4103
80%       5782
90%       8500
100%     29857
max      29857
Name: rmse, dtype: int64

In [28]:
final_results_df['price_diff'].abs().describe(percentiles=np.arange(0,1.1,0.1)).astype(int)

count    357794
mean        760
std        1751
min           0
0%            0
10%           3
20%          19
30%          52
40%         110
50%         206
60%         357
70%         603
80%        1032
90%        1955
100%      88805
max       88805
Name: price_diff, dtype: int64

In [29]:
overall_accuracy = (100 * (1- mean_absolute_percentage_error(final_results_df['price'], final_results_df['predicted_price']))).round(2)

In [30]:
f"Model overall accuracy: {overall_accuracy}%"

'Model overall accuracy: 92.37%'

In [31]:
overall_error = mean_absolute_error(final_results_df['price'], final_results_df['predicted_price']).astype(int)

In [32]:
f"Model overall price-vs-prediction difference: {overall_error}€"

'Model overall price-vs-prediction difference: 760€'

In [194]:
root_mean_squared_error(final_results_df['price'], final_results_df['predicted_price']).astype(int)

2215

In [181]:
# Save performance metrics to CSV
performance_df = pd.DataFrame(performance_metrics)
performance_df.to_csv('../06_app/app_files/performance_metrics.csv', index=False)

In [4]:
final_results_df.to_csv('../06_app/app_files/final_results_df.csv', index=False)

In [3]:
final_results_df = pd.read_csv('../06_app/app_files/final_results_df.csv')

In [184]:
performance_df = pd.read_csv('../06_app/app_files/performance_metrics.csv')